# Talmud Lab, Word Translation Project

A demonstration of the latest methods developed for the Talmud word-to-Jastrow project, which aims to map every word in the Talmud to a single entry (or ranked list of entries) in the Jastrow Talmudic dictionary.

In [1]:
from utils.texts import *
from utils.deconstruct2 import *
from utils.jastrow import *
from utils.respell import *

In [2]:
import re
from string import punctuation

import numpy as np
import pandas as pd
import random as rd

## Download a masekhet

`get_masekhet` downloads the full Hebrew text of a masekhet and tags the Steinsaltz "chunks" based on whether they are mishnayot or gemaras.

In [3]:
ber = Masekhet('Berakhot')

In [4]:
ber[:5]

[[['מֵאֵימָתַי',
   'קוֹרִין',
   'אֶת',
   'שְׁמַע',
   'בָּעֲרָבִין',
   '?',
   'מִשָּׁעָה',
   'שֶׁהַכֹּהֲנִים',
   'נִכְנָסִים',
   'לֶאֱכוֹל',
   'בִּתְרוּמָתָן',
   '.',
   'עַד',
   'סוֹף',
   'הָאַשְׁמוּרָה',
   'הָרִאשׁוֹנָה',
   '.',
   'דִּבְרֵי',
   'רַבִּי',
   'אֱלִיעֶזֶר',
   '.'],
  ['וַחֲכָמִים', 'אוֹמְרִים', ':', 'עַד', 'חֲצוֹת', '.'],
  ['רַבָּן',
   'גַּמְלִיאֵל',
   'אוֹמֵר',
   'עַד',
   'שֶׁיַּעֲלֶה',
   'עַמּוּד',
   'הַשַּׁחַר',
   '.'],
  ['מַעֲשֶׂה',
   'וּבָאוּ',
   'בָנָיו',
   'מִבֵּית',
   'הַמִּשְׁתֶּה',
   ',',
   'אָמְרוּ',
   'לוֹ',
   ':',
   'לֹא',
   'קָרִינוּ',
   'אֶת',
   'שְׁמַע',
   ',',
   'אָמַר',
   'לָהֶם',
   ':',
   'אִם',
   'לֹא',
   'עָלָה',
   'עַמּוּד',
   'הַשַּׁחַר',
   'חַיָּיבִין',
   'אַתֶּם',
   'לִקְרוֹת',
   '.',
   'וְלֹא',
   'זוֹ',
   'בִּלְבַד',
   'אָמְרוּ',
   ',',
   'אֶלָּא',
   'כָּל',
   'מַה',
   'שֶּׁאָמְרוּ',
   'חֲכָמִים',
   '״',
   'עַד',
   'חֲצוֹת',
   '״',
   ',',
   'מִצְוָתָן',
   'עַד',
   'שֶׁיַּעֲלֶה

In [5]:
len(ber)

125

## Removing prefixes

`detach_prefix` removes all possible prefixes from a word based on the rules of nikkud to get its root.

In [4]:
r_page = rd.randrange(len(ber))
r_chunk = rd.randrange(len(ber[r_page]))
selected = ber[r_page][r_chunk]
print(' '.join(selected))
words = [i for i in selected if not (i in punctuation + '…׳״—')]

words

תָּנָא אוֹתוֹ הַיּוֹם , סִלְּקוּהוּ לְשׁוֹמֵר הַפֶּתַח וְנִתְּנָה לָהֶם רְשׁוּת לַתַּלְמִידִים לִיכָּנֵס . שֶׁהָיָה רַבָּן גַּמְלִיאֵל מַכְרִיז וְאוֹמֵר : כָּל תַּלְמִיד שֶׁאֵין תּוֹכוֹ כְּבָרוֹ , לֹא יִכָּנֵס לְבֵית הַמִּדְרָשׁ .


['תָּנָא',
 'אוֹתוֹ',
 'הַיּוֹם',
 'סִלְּקוּהוּ',
 'לְשׁוֹמֵר',
 'הַפֶּתַח',
 'וְנִתְּנָה',
 'לָהֶם',
 'רְשׁוּת',
 'לַתַּלְמִידִים',
 'לִיכָּנֵס',
 'שֶׁהָיָה',
 'רַבָּן',
 'גַּמְלִיאֵל',
 'מַכְרִיז',
 'וְאוֹמֵר',
 'כָּל',
 'תַּלְמִיד',
 'שֶׁאֵין',
 'תּוֹכוֹ',
 'כְּבָרוֹ',
 'לֹא',
 'יִכָּנֵס',
 'לְבֵית',
 'הַמִּדְרָשׁ']

In [5]:
for w in words:
    print(w, end='\t')
    print(detach_prefix(w))

תָּנָא	['תָּנָא']
אוֹתוֹ	['אוֹתוֹ']
הַיּוֹם	['הַיּוֹם', 'יּוֹם']
סִלְּקוּהוּ	['סִלְּקוּהוּ']
לְשׁוֹמֵר	['לְשׁוֹמֵר', 'שׁוֹמֵר']
הַפֶּתַח	['הַפֶּתַח', 'פֶּתַח']
וְנִתְּנָה	['וְנִתְּנָה', 'נִתְּנָה']
לָהֶם	['לָהֶם']
רְשׁוּת	['רְשׁוּת']
לַתַּלְמִידִים	['לַתַּלְמִידִים']
לִיכָּנֵס	['לִיכָּנֵס', 'יכָּנֵס']
שֶׁהָיָה	['שֶׁהָיָה', 'הָיָה']
רַבָּן	['רַבָּן']
גַּמְלִיאֵל	['גַּמְלִיאֵל']
מַכְרִיז	['מַכְרִיז']
וְאוֹמֵר	['וְאוֹמֵר', 'אוֹמֵר']
כָּל	['כָּל']
תַּלְמִיד	['תַּלְמִיד']
שֶׁאֵין	['שֶׁאֵין', 'אֵין']
תּוֹכוֹ	['תּוֹכוֹ']
כְּבָרוֹ	['כְּבָרוֹ', 'בָרוֹ']
לֹא	['לֹא']
יִכָּנֵס	['יִכָּנֵס']
לְבֵית	['לְבֵית', 'בֵית']
הַמִּדְרָשׁ	['הַמִּדְרָשׁ', 'מִּדְרָשׁ']


## Getting roots

In [8]:
for w in words:
    print(w, end='\t')
    
    possibilities = detach_prefix(w)
    print(possibilities)
    
    print('Hebrew:')
    for p in possibilities:
        print(p + '\t' + str(hebrew_root(p)))
    
    print('Aramaic verb:')
    for p in possibilities:
        print(p + '\t' + str(aramaic_verb_root(p)))
    
    print('Aramaic noun:')
    for p in possibilities:
        print(p + '\t' + str(aramaic_noun_root(p)))
    
    print('\n\n')

תָּנָא	['תָּנָא']
Hebrew:
תָּנָא	[('תַּנָּא', 'שֵם')]
Aramaic verb:
תָּנָא	[]
Aramaic noun:
תָּנָא	[]



אוֹתוֹ	['אוֹתוֹ']
Hebrew:
אוֹתוֹ	[('אֶת', 'מילת יחס'), ('אוֹת', 'כינוי רומז לרחוק'), ('אַוָּה', 'שֵם')]
Aramaic verb:
אוֹתוֹ	[]
Aramaic noun:
אוֹתוֹ	[]



הַיּוֹם	['הַיּוֹם', 'יּוֹם']
Hebrew:
הַיּוֹם	[('הַיּוֹם', 'תואר הפועל'), ('יוֹם', 'שֵם')]
יּוֹם	[('יוֹם', 'שֵם')]
Aramaic verb:
הַיּוֹם	[]
יּוֹם	[]
Aramaic noun:
הַיּוֹם	[]
יּוֹם	[]



סִלְּקוּהוּ	['סִלְּקוּהוּ']
Hebrew:
סִלְּקוּהוּ	[('סִלֵּק', 'פועל', 'פיעל')]
Aramaic verb:
סִלְּקוּהוּ	[]
Aramaic noun:
סִלְּקוּהוּ	[]



לְשׁוֹמֵר	['לְשׁוֹמֵר', 'שׁוֹמֵר']
Hebrew:
לְשׁוֹמֵר	[('שׁוֹמֵר', 'שֵם'), ('שָׁמַר', 'פועל', 'קל'), ('שֻׁמָּר', 'שֵם')]
שׁוֹמֵר	[('שׁוֹמֵר', 'שֵם'), ('שָׁמַר', 'פועל', 'קל'), ('שֻׁמַּר', 'פועל', 'פועל'), ('שֻׁמָּר', 'שֵם')]
Aramaic verb:
לְשׁוֹמֵר	[]
שׁוֹמֵר	[]
Aramaic noun:
לְשׁוֹמֵר	[]
שׁוֹמֵר	[]



הַפֶּתַח	['הַפֶּתַח', 'פֶּתַח']
Hebrew:
הַפֶּתַח	[('פֶּתַח', 'שֵם'), ('פַּתָּח', 'שֵם'), ('נִפְתַּח', 'פועל', 'נפע

## Naive Jastrow Search

The following description has been pulled from the docs of the method used below:



`An extremely naive entry searcher, intended merely as a proof-of-concept, not for practical use.`

`The method uses naive lookup on different constructions of 'word' and returns the top N matching entries.`

`The "top N" are ranked by category:`

`1. In Jastrow directly.`

`2. Top entries from dicta nouns (since there are fewer nouns than verbs)`

`3. Top entries from dicta verbs`

`4. Top entries from Morfix.`

`The word is first put through respell.order_nikkud, and if more than one spelling is returned, the method returns`

`the top N for each spelling. Then its possible prefixes are removed, with the ranking choosing the shortest among`

`the possibilities as top rank.`

In [6]:
i = 1
for w in words:
    print(i, end='. ')
    print(w, end='\t')
    print(naive_top_n(w, 100))
    i += 1

1. תָּנָא	['תַּנָּא']
2. אוֹתוֹ	['אַתְרוֹגָא', 'אֵת', 'אוֹת I', 'אוֹת II', 'אוֹת III']
3. הַיּוֹם	['יוֹם', 'יוֹם ²']
4. סִלְּקוּהוּ	[]
5. לְשׁוֹמֵר	['שׁוֹמֵר', 'שָׁמַר']
6. הַפֶּתַח	['פֶּתַח', 'פַּתָּח', 'פָּתַח', 'פִּתֵּחַ']
7. וְנִתְּנָה	['נָתַן', 'תני']
8. לָהֶם	['לְ־']
9. רְשׁוּת	['רָשׁוּת I', 'רָשׁוּת II']
10. לַתַּלְמִידִים	['תַּלְמִיד']
11. לִיכָּנֵס	['כָּנַס']
12. שֶׁהָיָה	['הוי', 'שְׁהִיָּה']
13. רַבָּן	['רַבָּן ²', 'רַבָּן', 'רַב', 'רַב I', 'רַב II']
14. גַּמְלִיאֵל	['גַּמְלִיאֵל']
15. מַכְרִיז	['כָּרַז']
16. וְאוֹמֵר	['אָמַר I', 'אָמַר II', 'אֲמַר']
17. כָּל	['כָּל', 'כּוֹלְיָא', 'כַּלְּתָא', 'כֹּל']
18. תַּלְמִיד	['תַּלְמִיד']
19. שֶׁאֵין	['אֵין', 'אִין I', 'אַיִן II', 'אַיִן I', 'אִין II']
20. תּוֹכוֹ	['תּוֹךְ II', 'תּוֹךְ I', 'תָּוֶךְ']
21. כְּבָרוֹ	['בַּר', 'בַּר IV', 'בַּר I', 'בַּר II', 'בַּר III', 'בָּרָה', 'ברי II']
22. לֹא	['לֹא', 'לָא II', 'לָא III', 'לָא I']
23. יִכָּנֵס	['כָּנַס']
24. לְבֵית	['*לְבָא', 'לְבָא', 'בַּיִת', 'בֵּיתָא I', 'שַׁרְיֵי']
25. הַמִּדְרָשׁ	[

## Smarter Naive

In [7]:
i = 1
for w in words:
    print(i, end='. ')
    print(w, end='\t')
    print(smart_naive_search(w))
    i += 1

1. תָּנָא	['תַּנָּא']
2. אוֹתוֹ	['אַתְרוֹגָא', 'אֵת', 'אוֹת I']
3. הַיּוֹם	['יוֹם', 'יוֹם ²']
4. סִלְּקוּהוּ	[]
5. לְשׁוֹמֵר	['שׁוֹמֵר']
6. הַפֶּתַח	['פֶּתַח']
7. וְנִתְּנָה	['תני', 'נָתַן']
8. לָהֶם	['לְ־']
9. רְשׁוּת	['רָשׁוּת I', 'רָשׁוּת II']
10. לַתַּלְמִידִים	['תַּלְמִיד']
11. לִיכָּנֵס	['כָּנַס']
12. שֶׁהָיָה	['הוי', 'שְׁהִיָּה']
13. רַבָּן	['רַבָּן ²', 'רַבָּן']
14. גַּמְלִיאֵל	['גַּמְלִיאֵל']
15. מַכְרִיז	['כָּרַז']
16. וְאוֹמֵר	['אָמַר I', 'אָמַר II', 'אֲמַר']
17. כָּל	['כָּל', 'כּוֹלְיָא']
18. תַּלְמִיד	['תַּלְמִיד']
19. שֶׁאֵין	['אִין I', 'אֵין', 'אַיִן II']
20. תּוֹכוֹ	['תּוֹךְ II', 'תּוֹךְ I', 'תָּוֶךְ']
21. כְּבָרוֹ	['בַּר', 'בַּר IV', 'בַּר I']
22. לֹא	['לֹא']
23. יִכָּנֵס	['כָּנַס']
24. לְבֵית	['*לְבָא', 'לְבָא']
25. הַמִּדְרָשׁ	['מִדְרָשׁ']
